In [1]:
% matplotlib inline
import matplotlib.pyplot
matplotlib.rcParams['font.family'] = 'Arial,freesans'

import numpy
import pandas
import pathlib
import astropy.coordinates
from astropy.coordinates import AltAz
import astropy.units
import astropy.time
import astropy.constants

import pickle
import time
import os
import sys
import multiprocessing

import necstdb

sys.path.append("/home/amigos/git/analy_n2data/script")
#import coordinate_calc
import kisa_rev

In [2]:
tp_topics = [
        'xffts_power_board01',
        'xffts_power_board02',
        'xffts_power_board03',
        'xffts_power_board04',
        'xffts_power_board05', 
        'xffts_power_board06',
        'xffts_power_board07',
        'xffts_power_board08',
        'xffts_power_board09',
        'xffts_power_board10', 
        'xffts_power_board11',
        'xffts_power_board12',
        'xffts_power_board13', 
        'xffts_power_board14', 
        'xffts_power_board15',
        'xffts_power_board16',
]

board2hz = {
        'xffts_power_board01': '200beam',
        'xffts_power_board02': '200beam',
        'xffts_power_board03': '100beam',
        'xffts_power_board04': '100beam',
        'xffts_power_board05': '100beam', 
        'xffts_power_board06': '100beam',
        'xffts_power_board07': '100beam',
        'xffts_power_board08': '100beam',
        'xffts_power_board09': '100beam',
        'xffts_power_board10': '100beam', 
        'xffts_power_board11': '100beam',
        'xffts_power_board12': '100beam',
        'xffts_power_board13': '100beam', 
        'xffts_power_board14': '100beam', 
        'xffts_power_board15': '100beam',
        'xffts_power_board16': '100beam',
}

In [3]:
def open_necstdb(path):
    db = necstdb.opendb(path)
    
    tp = {_: db.open_table(_).read(astype='array') for _ in tp_topics}
    tables = {
        'tp':  tp,
        'tp_receivedtime': {_: tp[_]['received_time'][1:] - tp[_]['received_time'][:-1] for _ in tp},
        'tp_timestamp': {_: tp[_]['timestamp'][1:] - tp[_]['timestamp'][:-1] for _ in tp},
        'load': db.open_table('cpz7415v_1_rsw0_u_step').read(astype='array'),
        'enc': db.open_table('status_encoder').read(astype='array'),
        'obsmode': db.open_table('obsmode').read(astype='array'),
        'weather' : db.open_table('status_weather').read(astype='array')
    }
    
    return tables

def get_target_coord(enc, weather, body='sun'):
    _target = {}
    target = astropy.coordinates.get_body(
        body = body, 
        time = astropy.time.Time(
            enc['timestamp'], 
            format = 'unix',
        ), 
    )
    
    nanten2 = astropy.coordinates.EarthLocation(
        lon =  -67.70308139 * astropy.units.deg,
        lat = -22.96995611  * astropy.units.deg,
        height = 4863.85 * astropy.units.m,
    )

    target.location = nanten2
    
    _target['100beam'] = target.transform_to(AltAz(obstime=astropy.time.Time(enc['timestamp'], format = 'unix'),
                                                   pressure=weather['press'].mean()*astropy.units.hPa, 
                                                   obswl=(astropy.constants.c/(115*astropy.units.GHz)).to('micron'),
                                                   temperature=(weather['out_temp'].mean() - 273.15)*astropy.units.deg_C,
                                                   relative_humidity = weather['out_humi'].mean()/100
                                                  )
                                            )
    
    _target['200beam'] = target.transform_to(AltAz(obstime=astropy.time.Time(enc['timestamp'], format = 'unix'), 
                                                   pressure=weather['press'].mean()*astropy.units.hPa, 
                                                   obswl=(astropy.constants.c/(230*astropy.units.GHz)).to('micron'),
                                                   temperature=(weather['out_temp'].mean() - 273.15)*astropy.units.deg_C,
                                                   relative_humidity = weather['out_humi'].mean()/100
                                                  )
                                            )
    
    return _target


def apply_kisa(target, enc):
    kisa = {}
    paz ={}
    pel = {}
    daz = {}
    del_ = {}
    daz_del = {}
    for _ in target:
        kisa[_] = [
            kisa_rev.apply_kisa_test(
                a.az.rad,
                a.alt.rad,
                "/home/amigos/data/otf_planet2018/sun_scan/20191227/hosei/hosei_230.txt"
            )
    
            for a in target[_]
        ]
    
        kisa[_] = numpy.array(kisa[_]).T

        paz[_] = target[_].az.deg - kisa[_][0] / 3600
        pel[_] = target[_].alt.deg - kisa[_][1] / 3600
    
        if numpy.nanmean(paz[_]) > 180:
            paz[_] -= 360
        elif numpy.nanmean(paz[_]) < -180:
            paz[_] += 360
            pass

        daz[_] = paz[_] - enc['enc_az']/3600 
        del_[_] = pel[_] - enc['enc_el']/3600

        daz_del[_] = pandas.concat(
             [
                pandas.DataFrame(
                   daz[_],
                    index = pandas.to_datetime(enc['timestamp'], unit='s'),
                    columns = ['daz'],
                ),
            
                pandas.DataFrame(
                    del_[_],
                    index = pandas.to_datetime(enc['timestamp'], unit='s'),
                    columns = ['del'],
                ),
            ],
            axis = 1,
        )
    return daz_del

def interpret_obsmode(obsmode):
    modeint = numpy.zeros(len(obsmode)) -1
    modeint[obsmode['obs_mode'] == b'HOT       '] = 0
    modeint[obsmode['obs_mode'] == b'OFF       '] = 1
    modeint[obsmode['obs_mode'] == b'ON        '] = 2
    
    obsmodeint = pandas.DataFrame(
                modeint,
                index = pandas.to_datetime(obsmode['received_time'], unit='s'),
                columns = ['obsmode'],
    )
    return obsmodeint


def regrid(tables):
    df_ = {
        _ : pandas.concat(
            [
                pandas.DataFrame(
                    tables['enc']['enc_az'] / 3600,
                    index = pandas.to_datetime(tables['enc']['timestamp'], unit='s'),
                    columns = ['az'],
                ),
                 
                pandas.DataFrame(
                    tables['enc']['enc_el'] / 3600,
                    index = pandas.to_datetime(tables['enc']['timestamp'], unit='s'),
                    columns = ['el'],
                ),
              
                tables['dazel'][board2hz[_]]['daz'],
                tables['dazel'][board2hz[_]]['del'],
        
            
                pandas.DataFrame(
                    tables['tp'][_]['total_power'],
                    index = pandas.to_datetime(tables['tp'][_]['received_time'], unit='s'),
                    columns = ['tp'],
                ),
                
                tables['obsmode'],
            ],

            axis = 1,
        )
        for _ in tables['tp']
    }

    df = {_ : df_[_].resample('0.2S').median() for _ in df_}
    return df

In [4]:
def chopper_wheel(df):
    tphot = {
        _ : numpy.nanmean(df[_]['tp'][df[_]['obsmode']==0])
        for _ in df
    }

    tpoff = {
        _ : numpy.nanmean(df[_]['tp'][df[_]['obsmode']==1])
        for _ in df
    }

    tpon = {
        _ : df[_]['tp'][df[_]['obsmode']==2]
        for _ in df
    }
    
    ta = {
        _ : (tpon[_] - tpoff[_]) / (tphot[_] - tpoff[_]) * 300
        for _ in df
    }
        
    df_ = {
        _ : pandas.concat(
            [
                pandas.DataFrame(
                    ta[_].values,
                    index = tpon[_].index,
                    columns = ['ta'],
                ),
                 
                pandas.DataFrame(
                    df[_]['daz'][df[_]['obsmode']==2],
                    index = tpon[_].index,
                    columns = ['daz'],
                ),
              
                pandas.DataFrame(
                    df[_]['del'][df[_]['obsmode']==2],
                    index = tpon[_].index,
                    columns = ['del'],
                ),
              
                pandas.DataFrame(
                    df[_]['az'][df[_]['obsmode']==2],
                    index = tpon[_].index,
                    columns = ['az'],
                ),
              
                pandas.DataFrame(
                    df[_]['el'][df[_]['obsmode']==2],
                    index = tpon[_].index,
                    columns = ['el'],
                ),              
            ],

            axis = 1,
        )
        for _ in df
    }

    return df_


def writeto(df_dict, path):
    f = open(path, 'wb')
    pickle.dump(df_dict, f)
    f.close()
    return
    

In [5]:
saveto = '/home/amigos/data/otf_planet2018/sun_scan/20191227/chopper_data/'

def create_chopperwheeled_data(path, body='sun'):
    if path.name.find('sun') == -1: return
    #try:
    print(path)
    tables = open_necstdb(path)
    target = get_target_coord(tables['enc'], tables['weather'], body)
    tables['dazel'] = apply_kisa(target, tables['enc'])
    tables['obsmode'] = interpret_obsmode(tables['obsmode'])
    regrided_data = regrid(tables)
    cw_data = chopper_wheel(regrided_data)
    writeto(cw_data, saveto + path.name + '.pickle')
    print('writeto : %s'%(saveto + path.name + '.pickle'))
    #except:
      #  pass
    return

In [6]:
data_file = pathlib.Path('/home/amigos/nfs_hdd3/data/observation/otf_planet2018/20191227').iterdir()

with multiprocessing.Pool(processes=3) as pool:
    pool.map(create_chopperwheeled_data, data_file)
    pass

SyntaxError: invalid syntax (<ipython-input-6-b360cc271c61>, line 4)